In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
from chinese_calendar import is_workday, is_holiday
import chinese_calendar as calendar
import time, datetime
from  tqdm import tqdm
import numba
from numba import jit
warnings.filterwarnings('ignore')
text = pd.read_csv("PRSA_data.csv",encoding='unicode_escape').convert_dtypes()
text.isnull().sum()
text['pm2.5'].astype('float')


# one-hot encoding
text['cbwd'].unique()
text = text.join(pd.get_dummies(text['cbwd']).astype('float'))
del text['cbwd']

In [56]:

def generate_dataset(data):

    Train_set=pd.DataFrame(columns=data.columns)
    Test_set=pd.DataFrame(columns=data.columns)
    text_data =data.groupby(['year','month','day'])
    for index,(name,data)  in enumerate(text_data):
        if((index+1)%7 ==0):
            Test_set =Test_set.append(data)
        else:
            Train_set = Train_set.append(data)
    return Train_set,Test_set
Train_set ,Test_set = generate_dataset(data=text)



In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

def missForest(train_set):

    x_train =train_set.dropna()
    y_train = x_train.pop('pm2.5')

    #找到y是空的值;
    index =train_set[train_set['pm2.5'].isna()].index
    x_test =train_set.loc[index,:]
    x_test.pop('pm2.5')
    rfc = RandomForestRegressor(n_estimators=100)
    rfc = rfc.fit(x_train,y_train)
    Ypredict = rfc.predict(x_test)

    #返回填充好的值
    data  =train_set.copy()
    # data['pm2.5'] =data['pm2.5'].astype('float')
    data.loc[data['pm2.5'].isnull(),'pm2.5'] = Ypredict
    return data
def missKnn(train_set):
    data = train_set.copy()
    #将NA转化成nan
    data['pm2.5'] = pd.to_numeric(data['pm2.5'], errors='coerce')
    imputer = KNNImputer(n_neighbors=6)
    data_ = imputer.fit_transform(data)
    data_ =pd.DataFrame(data_,columns=train_set.columns)
    return data_

missforest_fill_in =missForest(Train_set)
missKnn_fill_in = missKnn(Train_set)
average_fill_in=Train_set.fillna(int(Train_set['pm2.5'].mean()))
median_fill_in=Train_set.fillna(int(Train_set['pm2.5'].median()))

In [ ]:
from xgboost import XGBRegressor
def get_score(trainset,testset):
    trainset =trainset.astype("float")
    testset =testset.astype("float")
    train_set=trainset.copy()
    test_set=testset.copy()
    Y_train =train_set['pm2.5']
    Y_test =test_set['pm2.5']
    del train_set['pm2.5']
    del test_set['pm2.5']
    X_train = train_set
    X_test =test_set

    model = XGBRegressor()
    model.fit(X_train,Y_train)
    score = model.score(X_test,Y_test)
    return score

t_s= Test_set.dropna()
train_s =Train_set.dropna()
Scores=[]
dropna_scores = get_score(train_s,t_s)
average_scores = get_score(average_fill_in,t_s)
median_scores = get_score(median_fill_in,t_s)
missKnn_scores = get_score(missKnn_fill_in,t_s)
foreset_scores = get_score(missforest_fill_in,t_s)
Scores.append(dropna_scores)
Scores.append(average_scores)
Scores.append(median_scores)
Scores.append(missKnn_scores)
Scores.append(foreset_scores)

plt.plot(range(1,6),Scores)


In [ ]:
Scores